<h3 style="text-align: center;"><b>NLP Final Exam</b></h3>


In [1]:
import torch
from torch import nn
from torch import functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# Дисклеймер про CrossEntropyLoss и NLLLoss

Обычно в PyTorch не нужно делать Softmax как последний слой модели. 

* Если Вы используете NLLLoss, то ему на вход надо давать лог вероятности, то есть выход слоя LogSoftmax. (Просто результат софтмакса, к которому применен логарифм)
* Если Вы используете CrossEntropyLoss, то применение LogSoftmax уже включено внутрь лосса, поэтому ему на вход надо подавать просто выход обычного линейного слоя без активации. По сути CrossEntropyLoss = LogSoftmax + NLLLoss

Зачем такие сложности, чтобы посчитать обычную кросс энтропию? Дело в том, что нам в любом случае придется взять логарифм от результатов софтмакса, а если делать это одной функцией, то можно сделать более устойчивую реализацию, которая даст меньшую вычислительную погрешность. 

Таким образом, если у вас в конце сети, решающей задачу классификации, стоит просто линейный слой без активации, то вам нужно использовать CrossEntropy. В notebook используется лосс CrossEntropy

# Задание 1. Создайте генератор батчей. 

В этот раз мы хотим сделать генератор, который будет максимально похож на то, что используется в реальном обучении. 

С помощью numpy вам нужно перемешать исходную выборку и выбирать из нее батчи размером batch_size, если размер выборки не делился на размер батча, то последний батч должен иметь размер меньше batch_size и состоять просто из всех оставшихся объектов. Возвращать нужно в формате (X_batch, y_batch). Необходимо написать именно генератор, то есть вместо return использовать yield. 

Хорошая статья про генераторы: https://habr.com/ru/post/132554/


**Ответ на задание - код**


In [30]:
def batch_generator(X, y, batch_size):
    np.random.seed(42)
    jerm = np.random.permutation(len(X))
    print(len(jerm))
    n_samplers = []
    for r in range(len(X)):
        n_samplers.append([X[r], y[r]])
        
    # YOUR CODE
    
    for r in range(0, len(jerm), batch_size):
        if r < batch_size:
            bat_samp = [n_samplers[jerm[r]], n_samplers[jerm[r+1]]]
            X_train = []
            y_train = []

            for batxrw_s in bat_samp:
                X_train.append(batxrw_s[0])
                y_train.append(batxrw_s[1])
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            print(X_train, y_train)
            yield X_train, y_train
        else:
            bat_samp = [n_samplers[jerm[r]]]
            X_train = []
            y_train = []

            for batx_s in bat_samp:
                X_train.append(batx_s[0])
                y_train.append(batx_s[1])

            X_train = np.array(X_train)
            y_train = np.array(y_train)
            print(X_train, y_train)
            yield X_train, y_train


Попробуем потестировать наш код

In [31]:
from inspect import isgeneratorfunction
assert isgeneratorfunction(batch_generator), "batch_generator должен быть генератором! В условии есть ссылка на доки"

X = np.array([
              [1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]
])
y = np.array([
              1, 2, 3
])

# Проверим shape первого батча
iterator = batch_generator(X, y, 2)
X_batch, y_batch = next(iterator)
assert X_batch.shape == (2, 3), y_batch.shape == (2,)
assert np.allclose(X_batch, X[:2]), np.allclose(y_batch, y[:2])

# Проверим shape последнего батча (их всего два)
X_batch, y_batch = next(iterator)
assert X_batch.shape == (1, 3), y_batch.shape == (1,)
assert np.allclose(X_batch, X[2:]), np.allclose(y_batch, y[2:])

# Проверим, что итерации закончились
iter_ended = False
try:
    next(iterator)
except StopIteration:
    iter_ended = True
assert iter_ended

# Еще раз проверим то, сколько батчей создает итератор
X = np.random.randint(0, 100, size=(1000, 100))
y = np.random.randint(-1, 1, size=(1000, 1))
num_iter = 0
for _ in batch_generator(X, y, 3):
    num_iter += 1
assert num_iter == (1000 // 3 + 1)

3
[[1 2 3]
 [4 5 6]] [1 2]
[[7 8 9]] [3]
1000
[[38 53 99 10 57 39  3 63 47 69 82 30  1 42 88 40 59 74 45 39 58 44 41 20
   2 65 10 23 49 87 20 24  3 42 46 22 59 87 20 65 86 50 14 57 25 89 91 58
  89 44 65 34 95 93 63 83 76 26 86 79 30 11 96 44 31 53 65 86 31 14 54 40
  39 71 48 85 37 55 33  0  9 39 15 87 45 23 13 89 38 17 83 40 55 35 23 89
  28 21 10 40]
 [81 66 11 57 19 49  7 30 23 12 71 41 77 78 84 78 25 37 37 54 96  9 82 10
   2 22 61 69 40 40 92  3  2 83 50 54 62 48 71 69 68  7  7 98 18 73 54 98
  72 71  7 56 71 83  6 45 21 80 43  4 97 24 46 42 14 63 30 82 43 94 46 11
  63 22 49 86 35 10 85 46 55 93 68 36 83  9 55 85 41 29 93 21 54 37 55 70
  35 77 10  6]] [[-1]
 [-1]]
[[70 66 99 30 24 46 55 87  0 94 36 74 66 11 19 29 73 63 78 84 63 17 31 18
  93 32 61 21 28 87  3 49 75 99  3  4 82 38 56 30  6 11  9 28 84 12 86 69
  30  8 50 87 15 58 57 10 35  0 20 43 99 36  5 13 92  0  7 43 82 86 74 26
  35 41 19  7 45 99 51 25 34 59  2  3 92 61 63 49 91 18 42 91 81 13 13 53
  28  2 97 72]] [[0]]


# Задание 2. Обучите модель для классификации звезд

Загрузите датасет из файла sky_data.csv, разделите его на train/test и обучите на нем нейронную сеть (архитектура ниже). Обучайте на батчах с помощью оптимизатора Adam, lr подберите сами, пробуйте что-то вроде 1e-2

Архитектура:

1. Dense Layer с relu активацией и 50 нейронами
2. Dropout 80% (если другой keep rate дает сходимость лучше, то можно изменить) (попробуйте 50%) 
3. BatchNorm
4. Dense Layer с relu активацией и 100 нейронами
5. Dropout 80% (если другой keep rate дает сходимость лучше, то можно изменить) (попробуйте для разнообразия 50%)
6. BatchNorm
7. Выходной Dense слой c количеством нейронов, равному количеству классов

Лосс - CrossEntropy.

В датасете классы представлены строками, поэтому классы нужно закодировать. Для этого в строчке ниже объявлен dict, с помощью него и функции map превратите столбец с таргетом в целое число. Кроме того, за вас мы выделили признаки, которые нужно использовать.

### Загрузка и обработка данных

In [46]:
feature_columns = ['ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'run', 'camcol', 'field']
target_column = 'class'

target_mapping = {
    'GALAXY': 0,
    'STAR': 1,
    'QSO': 2
}

In [47]:
data = pd.read_csv('https://drive.google.com/uc?id=1K-8CtATw6Sv7k2dXco1fL5MAhTbKtIH3')
data['class'].value_counts()

GALAXY    4998
STAR      4152
QSO        850
Name: class, dtype: int64

In [48]:
data.head()

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,STAR,-0.000009,3306,54922,491
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,STAR,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,GALAXY,0.123111,287,52023,513
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,4,269,3.722370e+18,STAR,-0.000111,3306,54922,510
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,4,269,3.722370e+18,STAR,0.000590,3306,54922,512


In [56]:
# Extract Features
X = data[feature_columns]
# Extract target
y = data[target_column]

# encode target with target_mapping
y = y.replace({'class': target_mapping})
print(y)

0         STAR
1         STAR
2       GALAXY
3         STAR
4         STAR
         ...  
9995    GALAXY
9996    GALAXY
9997      STAR
9998    GALAXY
9999    GALAXY
Name: class, Length: 10000, dtype: object


Нормализация фичей

In [60]:
# Просто вычтите среднее и поделитe на стандартное отклонение (с помощью пандас). Также преобразуйте всё в np.array
X = X.mean()/X.std(ddof=0);

AttributeError: 'float' object has no attribute 'to_numpy'

In [59]:
assert type(X) == np.ndarray and type(y) == np.ndarray, 'Проверьте, что получившиеся массивы являются np.ndarray'
assert np.allclose(y[:5], [1,1,0,1,1])
assert X.shape == (10000, 10)
assert np.allclose(X.mean(axis=0), np.zeros(10)) and np.allclose(X.std(axis=0), np.ones(10)), 'Данные не отнормированы'


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


AssertionError: Проверьте, что получившиеся массивы являются np.ndarray

Обучение

In [0]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# Превратим данные в тензоры, чтобы потом было удобнее
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

Хорошо, данные мы подготовили, теперь надо объявить модель

In [0]:
torch.manual_seed(42) 
np.random.seed(42)
model = nn.Sequential(
    <YOUR CODE>
)
    
loss_fn = <YOUR CODE>
optimizer = <YOUR CODE>

### Обучающий цикл

In [0]:
def train(X_train, y_train, X_test, y_test, num_epoch):
    train_losses = []
    test_losses = []
    for i in range(num_epoch):
        epoch_train_losses = []
        for X_batch, y_batch in batch_generator(X_train, y_train, 500):
            # На лекции мы рассказывали, что дропаут работает по-разному во время обучения и реального предсказания
            # Чтобы это учесть нам нужно включать и выключать режим обучения, делается это командой ниже
            model.train(True)
            # Посчитаем предсказание и лосс
            # YOUR CODE
            
            # зануляем градиент
            # YOUR CODE
            
            # backward
            # YOUR CODE
            
            # ОБНОВЛЯЕМ веса
            # YOUR CODE
            
            # Запишем число (не тензор) в наши батчевые лоссы
            epoch_train_losses.append(# YOUR CODE)        
        train_losses.append(np.mean(epoch_train_losses))
        
        # Теперь посчитаем лосс на тесте
        model.train(False)
        with torch.no_grad():
            # Сюда опять же надо положить именно число равное лоссу на всем тест датасете
            test_losses.append(# YOUR CODE)
            
    return train_losses, test_losses

In [0]:
def check_loss_decreased():
    print("На графике сверху, точно есть сходимость? Точно-точно? [Да/Нет]")
    s = input()
    if s.lower() == 'да':
        print("Хорошо!")
    else:
        raise RuntimeError("Можно уменьшить дропаут, уменьшить lr, поправить архитектуру, etc")

In [0]:
train_losses, test_losses = train(# YOUR CODE) #Подберите количество эпох так, чтобы график loss сходился
plt.plot(range(len(train_losses)), train_losses, label='train')
plt.plot(range(len(test_losses)), test_losses, label='test')
plt.legend()
plt.show()
    
check_loss_decreased()
assert train_losses[-1] < 0.3 and test_losses[-1] < 0.3

### Вычислите accuracy получившейся модели на train и test

In [0]:
from sklearn.metrics import accuracy_score

model.eval()
train_pred_labels = #YOUR CODE: use forward
test_pred_labels = #YOUR CODE: use forward

train_acc = accuracy_score(# YOUR CODE)
test_acc = accuracy_score(# YOUR CODE)

assert train_acc > 0.9, "Если уж классифицировать звезды, которые уже видел, то не хуже, чем в 90% случаев"
assert test_acc > 0.9, "Новые звезды тоже надо классифицировать хотя бы в 90% случаев"

print("Train accuracy: {}\nTest accuracy: {}".format(train_acc, test_acc))

# Задание 3. Исправление ошибок в архитектуре

Только что вы обучили полносвязную нейронную сеть. Теперь вам предстоит проанализировать архитектуру нейронной сети ниже, исправить в ней ошибки и  обучить её с помощью той же функции train. Пример исправления ошибок есть в семинаре Григория Лелейтнера.

Будьте осторожнее и убедитесь, что перед запуском train вы вновь переопределили все необходимые внешние переменные (train обращается к глобальным переменным, в целом так делать не стоит, но сейчас это было оправдано, так как иначе нам пришлось бы передавать порядка 7-8 аргументов).

Чтобы у вас получилась такая же архитектура, как у нас, и ответы совпали, давайте определим некоторые правила, как исправлять ошибки:

1. Если вы видите лишний нелинейный слой, который стоит не на своем месте, просто удалите его. (не нужно добавлять новые слои, чтобы сделать постановку изначального слоя разумной. Удалять надо самый последний слой, который все портит. Для линейных слоев надо что-то исправить, а не удалить его)
2. Если у слоя нет активации, то добавьте ReLU или другую подходящую активацию
3. Если что-то не так с learning_rate, то поставьте 1e-2
4. Если что-то не так с параметрами, считайте первый параметр, который появляется, как верный (т.е. далее в сети должен использоваться он).
5. Ошибки могут быть и в полносвязных слоях. 



Задача все та же - классификация небесных объектов на том же датасете. После исправления сети вам нужно обучить ее.

**Ответ на задачу - средний лосс на тестовом датасете**

In [64]:
torch.manual_seed(42) 
np.random.seed(42)
model = nn.Sequential(
    nn.Linear(10, 50),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.BatchNorm1d(50),
    nn.Linear(50,100),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.BatchNorm1d(100),
    nn.Linear(100, 3),
    nn.ReLU()
)
print(model)
    
loss_fn = nn.CrossEntropyLoss()
#loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

Sequential(
  (0): Linear(in_features=10, out_features=50, bias=True)
  (1): Dropout(p=0.5, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=50, out_features=100, bias=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=100, out_features=3, bias=True)
  (9): ReLU()
)


NameError: name 'optim' is not defined

In [0]:
# RIGHT ARCH
torch.manual_seed(42)   
np.random.seed(42)
model = nn.Sequential(
    <YOUR CODE>
)


loss_fn = <YOUR CODE>
optimizer = <YOUR CODE>

### Обучите и протестируйте модель так же, как вы это сделали в задаче 2. Вычислите accuracy.

In [63]:
def train(X_train, y_train, X_test, y_test, num_epoch):
    train_losses = []
    test_losses = []
    for i in range(num_epoch):
        print("Epoch:", i)
        epoch_train_losses = []
        for X_batch, y_batch in batch_generator(X_train, y_train, 500):
            # На лекции мы рассказывали, что дропаут работает по-разному во время обучения и реального предсказания
            # Чтобы это учесть нам нужно включать и выключать режим обучения, делается это командой ниже
            model.train(True)
            # Посчитаем предсказание и лосс
            output = model.forward(X_batch)
            
            # зануляем градиент
            model.zero_grad()
            
            # backward
            loss = loss_fn(output, y_batch)
            loss.backward()
            
            # ОБНОВЛЯЕМ веса
            optimizer.step()
            
            # Запишем число (не тензор) в наши батчевые лоссы
            epoch_train_losses.append(loss.item())
        train_losses.append(np.nanmean(epoch_train_losses))
        
        # Теперь посчитаем лосс на тесте
        model.train(False)
        
        output = model.forward(X_test)
        loss = loss_fn(output, y_test)
        
        with torch.no_grad():
            # Сюда опять же надо положить именно число равное лоссу на всем тест датасете
            test_losses.append(loss.item())
            
    return train_losses, test_losses

train_acc = <YOUR CODE>
test_acc = <YOUR CODE>


assert train_acc > 0.9, "Если уж классифицировать звезды, которые уже видел, то не хуже, чем в 90% случаев"
assert test_acc > 0.9, "Новые звезды тоже надо классифицировать хотя бы в 90% случаев"


SyntaxError: invalid syntax (<ipython-input-63-add963b67dc2>, line 40)

## Задание 4. Опишите в чем заключались ошибки

## Задание 5. Опишите модель лематизации 
Необходимо описать пошагово на примере реализацию с помощью архитектуры Transformer состоящей из Encoder и Decoder

In [ ]:
Нейронные сети с трансформаторной архитектурой могут использоваться для различных задач НЛП. Однако основные задачи в «обучении» состоят в том, чтобы восстановить пропущенные слова в одном предложении и оценить логическую последовательность одного предложения из другого. По какой-то причине векторы длиной 512 используются для кодирования предложений или их пар (далее именуемых «текст»). (Кстати, Google недавно опубликовал пример использования BERT в TF HUB, а длина вектора там составляет 128). Представление текста состоит из трех векторов длиной 512 и происходит следующим образом:

Вектор token_input из позиции 0 содержит индексы токенов, остальные элементы заполнены нулями. Жетоны могут пониматься как словоформы, фразы и части словоформ, например префиксы, корни или окончания. Например, если текст разделен на 10 «токенов», первые 10 позиций в векторе token_input заполняются индексами «токенов» из словаря, а остальные - нулями.

Вектор mask_input кодирует пропущенные слова (так называемые «маски» [MASK]): если определенный «токен» скрыт маской, мы записываем один в соответствующую ячейку и заполняем оставшиеся ячейки нулями.

Вектор seg_input кодирует, какие токены принадлежат первому предложению, а какие - второму: Если токен принадлежит второму предложению, мы записываем один в соответствующую ячейку и заполняем оставшиеся ячейки нулями.